In [ ]:
import os
import re
import unicodedata
import numpy as np
import pandas as pd
import warnings
import nltk
import spacy
import en_core_web_md
import matplotlib.pyplot as plt
import plotly.express as px
from nltk.stem import WordNetLemmatizer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import euclidean_distances
from collections import defaultdict

In [ ]:
df = pd.read_csv('./Data/genData/text_sentences_words.csv') # this file has the texts with punctuation!
# remove [stuff] in between square brackets - should already be gone now
def remove_bracket(text):
    return re.sub('(\[[^w]*\]\s)', '',text)
df['text'] = df['text'].apply(remove_bracket)

In [ ]:
date1='2012-07-21'
source1='oba'
text1=df.query('date==@date1 and source==@source1')
text1

In [ ]:
date2='2010-07-26'
source2='wsj'
text2=df.query('date==@date2 and source==@source2')
text2

## POS

In [ ]:
# Full list of processors.... only need tokenize and pos for now
#============================
#| Processor    | Package   |
#----------------------------
#| tokenize     | combined  |
#| pos          | combined  |
#| lemma        | combined  |
#| depparse     | combined  |
#| sentiment    | sstplus   |
#| constituency | wsj       |
#| ner          | ontonotes |
#============================

In [ ]:
import stanza
stanza.download('en')
nlp_st = stanza.Pipeline('en', processors='tokenize, lemma, pos')

In [ ]:
def cumulative_count(items):
    '''items: list of objects to count
       returns: dictionary with key:item, value: cumulative count of each item'''
    counts = defaultdict(int)
    cumulative_counts = {}
    cumulative_count = 0
    
    for item in items:
        counts[item] += 1
        cumulative_count += 1
        cumulative_counts[item] = cumulative_count
        
    return(cumulative_counts)

In [ ]:
# OK Lets put it all together and loop over all speeches and count up the POS
# The nlp(text) uses a lot of gpu memory, causes errors sometimes, may need to restart notebook to freshen things up
pos_dicts = []
def get_pos(text):
    doc = nlp_st(text) # Run stanza on each speech
    mat_of_pos = [[word.pos for word in sentence.words] for sentence in doc.sentences] # matrix of POS for each sentence
    # How to flatten a list = [item for sublist in list_of_lists for item in sublist]
    list_of_pos = [pos for sentence in mat_of_pos for pos in sentence] # flatten matrix into one list of all pos
    pos_counts = cumulative_count(list_of_pos) # get dictionary with key: pos, value: count of that POS in speech
    sum1=sum(pos_counts.values())
    pos_prop = pos_counts.values()/sum1
    print(pos_prop)
get_pos(text1.loc[37, 'text'])

In [ ]:
doc = nlp_st(text1.loc[37, 'text']) # Run stanza on each speech
mat_of_pos = [[word.pos for word in sentence.words] for sentence in doc.sentences] # matrix of POS for each sentence
# How to flatten a list = [item for sublist in list_of_lists for item in sublist]
list_of_pos = [pos for sentence in mat_of_pos for pos in sentence] # flatten matrix into one list of all pos
pos_counts = cumulative_count(list_of_pos) # get dictionary with key: pos, value: count of that POS in speech
sum1=sum(pos_counts.values())

In [ ]:
### pick up here later and fix the previous two cells